In [1]:
import open3d as o3d 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import struct
import sys
import pandas as pd

from mpl_toolkits.mplot3d import Axes3D 
from matplotlib import cm
from pyntcloud import PyntCloud

In [2]:
input_folder = "../../voxel_data/single/"
f = "400_cropped.ply"
x, y, z = 64, 64, 16

In [9]:
pcd = o3d.io.read_point_cloud(os.path.join(input_folder, f))
downpcd = pcd.voxel_down_sample(voxel_size=0.4)
color = np.asarray(downpcd.colors)
cloud = PyntCloud.from_instance("open3d", downpcd)
voxelgrid_id = cloud.add_structure("voxelgrid", n_x=x, n_y=y, n_z=z, size_x=1.5, size_y=1.5, size_z=1.0,regular_bounding_box=False)
voxelgrid = cloud.structures[voxelgrid_id]

x_cords = voxelgrid.voxel_x
y_cords = voxelgrid.voxel_y
z_cords = voxelgrid.voxel_z
# import pdb;pdb.set_trace()
voxel = np.zeros((x, y, z)).astype(np.bool)

for x, y, z in zip(x_cords, y_cords, z_cords):
    voxel[x][y][z] = True
# voxels, voxelgrid, pcd_new = voxelize(pcd, 64, 64, 16)

voxelgrid_id_128 = cloud.add_structure("voxelgrid", n_x=128, n_y=128, n_z=16, regular_bounding_box=False)
voxelgrid_128 = cloud.structures[voxelgrid_id_128]

IndexError: index 63 is out of bounds for axis 0 with size 9

In [4]:
voxelgrid.colors.shape

(61786, 3)

In [5]:
voxelgrid.plot(d=3, mode="binary", cmap="Oranges")
voxelgrid_128.plot(d=3, mode="binary", cmap="Oranges")

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(51.27242660522461, 132.92689514160156, 26.76…

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(74.13211059570312, 190.23076248168945, 37.22…

In [10]:
feat = voxelgrid.get_feature_vector('binary')

In [11]:
scaled_shape = np.asarray(voxelgrid.shape) / min(voxelgrid.shape)
scaled_shape

array([1.46257542, 1.4125462 , 1.        ])

In [12]:
np.save("../../voxel_data/voxel_cropped_400_128", voxel)

In [13]:
# test
test_npy = np.load("../../voxel_data/400-data-new.npz")
gt = test_npy['gt'][0]
pred = test_npy['pred'][0]

# binary mode
pred[pred > 0.5] = 1
pred[pred < 0.5] = 0

x, y, z = pred.shape
n_size = pred.flatten().shape[0]

In [14]:
sys.path.insert(0,'/home/shrisha/masters/WS-20/AVG/data/pyntcloud')
from pyntcloud import *

# color
# voxel_colors = np.full((n_size, 3), (0,0,1), dtype=np.int)
voxel_colors = np.full((n_size, 3), (0, 0, 1), dtype=np.float)
voxel_colors_gt = np.full((n_size, 3), (0, 1, 1), dtype=np.float)
# voxel_colors = color
voxel_centers = (np.argwhere(pred) * scaled_shape).astype(np.float32)
plot.voxelgrid.plot_voxelgrid_with_pythreejs(voxel_centers, voxel_colors, width=800, height=500)
voxel_centers_gt = (np.argwhere(gt) * scaled_shape).astype(np.float32)
plot.voxelgrid.plot_voxelgrid_with_pythreejs(voxel_centers_gt, voxel_colors_gt, width=800, height=500)



/home/shrisha/.virtualenvs/avg/lib/python3.6/site-packages/pythreejs/traits.py:203: UserWarning: 64-bit data types not supported for WebGL data, casting to 32-bit.
  warnings.warn('64-bit data types not supported for WebGL '


Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(50.4239387512207, 133.01897811889648, 24.900…

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(51.27242660522461, 132.92689514160156, 26.76…

In [ ]:
pred = test_npy['pred'][0]
(pred.max() + pred.min())/2